In [34]:
cs_fission_pu_239 = [0.9, 0.06, 0.006]

cs_capture_pu_239 = [0.2, 0.04, 0.006]

cs_scattering_pu_239 = [[2.0, 0.0, 0.0], [0.275, 0.6, 0.0], [0.033, 0.171, 0.024]]

cs_total_pu_239 = [3.10, 0.975, 0.24]

cs_production_neutrons_pu_239 = [2., 2.5, 3.]

cs_xi_table_pu_239 = [0.0, 0.04, 0.96]


pu_239 = Sig(3, cs_fission_pu_239, cs_capture_pu_239, cs_scattering_pu_239, 
                      cs_production_neutrons_pu_239,  cs_xi_table_pu_239, cs_total_pu_239)


energy_groups_2 = [0, 2 * 1E10]

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [36]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_scattering_matrix, number_of_production_neutrons,
                 cs_xi_table, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = []
        self.sig_scattering_matrix = sig_scattering_matrix

        for i in range(0, len(self.sig_scattering_matrix)):
            self.sig_s.append(sum(self.sig_scattering_matrix[i]))
            
        self.number_of_production_neutrons = number_of_production_neutrons
        self.cs_xi_table = cs_xi_table
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_number_of_groups(self):
        
        return self.number_groups
        

In [37]:
energy_groups_2 = [0, 6.25, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [38]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0.
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        self.energy_group = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self): 
        
        return self.energy_group
     #   res = next(x for x, val in enumerate(self.energy_groups)
      #                            if val > self.energy)        
       # return res - 1
    
    def set_energy_group(self, energy_group):
        
        self.energy_group = energy_group       
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [39]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [40]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = []
        for i in range(0, self.xs[0].number_groups):
            energy_group_maximum = -1
            for xs in self.xs:
                energy_group_maximum = max(energy_group_maximum, xs.sig_t[i])
            delta_xs.append(energy_group_maximum)
            
        self.delta_xs = delta_xs                          
            
    def get_delta_xs(self, energy_group):
        
        return self.delta_xs[energy_group]
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [41]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [42]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                energy_group = random.choice([0, 1])
                current_particle.set_energy_group(energy_group)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [43]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [44]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [45]:
class Multigroup_estimator:
    def __init__(self):
        self.groups_before = []
        self.weights_before = []
        self.groups_after = []
        self.weights_after = []
        
    def get_flux_ration_n_m(self, first, secound):
        
        number_before = 0.
        number_after = 0.
        for i in range(0, len(self.groups_before)):
            
            if self.groups_before[i] == first:
                number_before += self.weights_before[i]           
                if self.groups_after[i] == secound:
                    number_after += self.weights_after[i]
        
        flux_ratio = number_after / number_before
    
        return flux_ratio   

In [46]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [47]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs / virtual_cs < random_number:
        return True
    else:
        return False

In [48]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [49]:
def process_scattering(particle, c_s):
    
    energy_group_idx = particle.get_energy_group()   
    
    energy_group_probability = []
    energy_scattering_matrix = c_s.sig_scattering_matrix[energy_group_idx]
    
    majorant = sum(energy_scattering_matrix)
    for i in range(0, len(energy_scattering_matrix)):
        energy_group_probability.append(energy_scattering_matrix[i] / majorant)
        
    energy_groups_idxes = list(range(0, len(energy_scattering_matrix)))
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=energy_group_probability)
    
    particle.set_energy_group(energy_group_idx)     

In [50]:
def process_fission(particle, c_s):
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]
    particle.set_particle_fission(number_of_production_neutrons)
    particle.set_terminated()
    energy_groups_idxes = list(range(0, c_s.number_groups))
    xi_matrix = c_s.cs_xi_table
    energy_group_idx = np.random.choice(energy_groups_idxes,  p=xi_matrix)
    particle.set_energy_group(energy_group_idx)
    

In [51]:
def process_real_collision(particle, free_path, c_s, flux_group_estimator):
    
    energy_group_idx = particle.get_energy_group()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':
        process_scattering(particle, c_s)

        
    if type_collision == 'fission':    
        process_fission(particle, c_s)
        

In [52]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [53]:
def process_one_particle_history(particle, universe, estimators, flux_estimator):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    
    while not particle.is_terminated():
        
        energy_group_before = particle.get_energy_group()
        weight_before = particle.get_weight()
        

        current_xs = universe.get_xs_by_coordinates(particle)
        energy_group_idx = particle.get_energy_group()
        delta_xs = universe.get_delta_xs(energy_group_idx)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        flux_estimator.groups_before.append(particle.get_energy_group())
        flux_estimator.weights_before.append(particle.get_weight())
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions 
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs, flux_estimator)
            
    
    return particle, sum_collisions

In [54]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [55]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])  

In [56]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [57]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [58]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [59]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [60]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [61]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [62]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [63]:
class Flux_group_universe_estimator:
    def __init__(self, group):
        self.group = group
        self.collision_sum = [0]
        
    def add_collision(self, particle):
            self.collision_sum[-1]  += particle.get_weight()    

In [64]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    flux_group_estimators = []
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    number_of_groups = universe.xs[0].get_number_of_groups()
    for i in range(0, number_interations):
        print("i == " + str(i))
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []
        flux_group_estimator = Multigroup_estimator()
        for j in range(0, batch_size):
            particle = initial_sources[j]           
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, 
                                                                            estimators, flux_group_estimator)
            batch_particles.append(terminate_particle)
            
            
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        flux_group_estimators.append(flux_group_estimator)
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s)              

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)
     
    return k_effective, k_effective_exp, k_effective_std, estimators, flux_group_estimators

In [65]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0, -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective", "reflective", "reflective", "reflective", "reflective"]


slab.set_boundaries_type(boundaries_type)
slab.set_box_sizes(-1000, -1000., -1000, 1000, 1000., 1000 )
slab.set_zero_point(0., 0., 0.)

In [66]:
cells = [slab]
materials = [pu_239]
U235_Reactor = Universe(cells, materials)

In [67]:
zero_group_estimator = Flux_group_estimator(0)
first_group_estimator = Flux_group_estimator(1)
third_group_estimator = Flux_group_estimator(2)
estimators = [zero_group_estimator, first_group_estimator, third_group_estimator]

In [68]:
test_number_of_particles = 10
test_number_interations = 1000
test_number_inactive = 200

In [69]:
k_effective,k_effective_exp, k_effective_std, estimators_result, flux_group_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, pu_239, estimators)

 num1  6
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
i == 103
i == 104
i == 105
i == 106
i == 107
i == 108
i == 109
i == 110
i == 111
i == 112
i == 113
i == 114
i == 115
i == 116
i == 117
i == 118
i == 119
i == 120
i == 121
i =

 keff_cycle , k_effective_exp, std_k_effective 1.610937265886947   1.6021526150962462  0.003466200852329768
i == 258
 keff_cycle , k_effective_exp, std_k_effective 1.6628161731915263   1.603198538511682  0.0035628933984900675
i == 259
 keff_cycle , k_effective_exp, std_k_effective 1.6218283591908151   1.6035142981842097  0.003516191314163488
i == 260
 keff_cycle , k_effective_exp, std_k_effective 1.5836943301839126   1.6031839653842048  0.003472837610844755
i == 261
 keff_cycle , k_effective_exp, std_k_effective 1.598244650420931   1.6031029930077578  0.0034163910861791215
i == 262
 keff_cycle , k_effective_exp, std_k_effective 1.5967504486750843   1.6030005326152954  0.003362397779736356
i == 263
 keff_cycle , k_effective_exp, std_k_effective 1.5861269771375262   1.6027326984013626  0.0033194189682030596
i == 264
 keff_cycle , k_effective_exp, std_k_effective 1.5546289031373888   1.6019810766003628  0.003352483880908609
i == 265
 keff_cycle , k_effective_exp, std_k_effective 1.6172250

 keff_cycle , k_effective_exp, std_k_effective 1.6284073326555781   1.5998833983555538  0.0022717348103385355
i == 328
 keff_cycle , k_effective_exp, std_k_effective 1.6269497351816449   1.6000948541120077  0.0022638143503406934
i == 329
 keff_cycle , k_effective_exp, std_k_effective 1.6318577933841971   1.600341078447451  0.0022596519009559337
i == 330
 keff_cycle , k_effective_exp, std_k_effective 1.5799720186348694   1.6001843933719695  0.002247670498369746
i == 331
 keff_cycle , k_effective_exp, std_k_effective 1.6147177559533556   1.6002953350710643  0.0022332041135444893
i == 332
 keff_cycle , k_effective_exp, std_k_effective 1.6027316977682116   1.6003137923642243  0.002216298183633031
i == 333
 keff_cycle , k_effective_exp, std_k_effective 1.6428807873800626   1.6006338449583284  0.0022227341218356935
i == 334
 keff_cycle , k_effective_exp, std_k_effective 1.6078237266749005   1.600687500792034  0.0022067365990877373
i == 335
 keff_cycle , k_effective_exp, std_k_effective 1.606

 keff_cycle , k_effective_exp, std_k_effective 1.5987666944420948   1.6005931091221566  0.001718667595644742
i == 398
 keff_cycle , k_effective_exp, std_k_effective 1.5929141616015519   1.600554326558921  0.0017104051692513993
i == 399
 keff_cycle , k_effective_exp, std_k_effective 1.5932341105428762   1.6005175415538153  0.001702185980389239
i == 400
 keff_cycle , k_effective_exp, std_k_effective 1.5996718077708565   1.6005133128849005  0.001693658945111479
i == 401
 keff_cycle , k_effective_exp, std_k_effective 1.608162248727546   1.6005513672920777  0.0016856413214313747
i == 402
 keff_cycle , k_effective_exp, std_k_effective 1.579726251447797   1.6004482726591853  0.0016804411999874632
i == 403
 keff_cycle , k_effective_exp, std_k_effective 1.5858508425585784   1.600376364136522  0.0016736881306074908
i == 404
 keff_cycle , k_effective_exp, std_k_effective 1.5922733898928376   1.6003366436745434  0.0016659371582443282
i == 405
 keff_cycle , k_effective_exp, std_k_effective 1.624749

 keff_cycle , k_effective_exp, std_k_effective 1.5977408356189837   1.5993392935553834  0.0014681808621689794
i == 468
 keff_cycle , k_effective_exp, std_k_effective 1.5824674291788017   1.5992763388375604  0.0014640464845296051
i == 469
 keff_cycle , k_effective_exp, std_k_effective 1.608590708148738   1.5993109647457802  0.0014590047169822276
i == 470
 keff_cycle , k_effective_exp, std_k_effective 1.6095918725602152   1.59934904218213  0.00145408959351585
i == 471
 keff_cycle , k_effective_exp, std_k_effective 1.6178382757019953   1.5994172681360779  0.0014503196447029764
i == 472
 keff_cycle , k_effective_exp, std_k_effective 1.554529082321509   1.5992522380411713  0.0014543712141362646
i == 473
 keff_cycle , k_effective_exp, std_k_effective 1.614359498090154   1.5993075759900686  0.001450090334574357
i == 474
 keff_cycle , k_effective_exp, std_k_effective 1.5939385249483646   1.5992879809132743  0.0014449212151120226
i == 475
 keff_cycle , k_effective_exp, std_k_effective 1.6357680

...

 keff_cycle , k_effective_exp, std_k_effective 1.6019508500610629   1.5996974818608412  0.0011223586162202237
i == 678
 keff_cycle , k_effective_exp, std_k_effective 1.6071667135144332   1.5997131078684845  0.0011201171237054705
i == 679
 keff_cycle , k_effective_exp, std_k_effective 1.621639921465807   1.599758884097289  0.001118713171532207
i == 680
 keff_cycle , k_effective_exp, std_k_effective 1.5728098977506333   1.5997027403757333  0.001117790952934592
i == 681
 keff_cycle , k_effective_exp, std_k_effective 1.6031391664625467   1.5997098847127122  0.001115487521317522
i == 682
 keff_cycle , k_effective_exp, std_k_effective 1.575525958174692   1.5996597105912642  0.001114301005403889
i == 683
 keff_cycle , k_effective_exp, std_k_effective 1.5787121846125018   1.5996163409722606  0.0011128369949597809
i == 684
 keff_cycle , k_effective_exp, std_k_effective 1.6163036649048677   1.59965081891427  0.0011110704410334125
i == 685
 keff_cycle , k_effective_exp, std_k_effective 1.64445273

 keff_cycle , k_effective_exp, std_k_effective 1.5807187075225613   1.5992522013244408  0.00105210801003689
i == 748
 keff_cycle , k_effective_exp, std_k_effective 1.5970287342121772   1.5992481439027029  0.0010501941879138327
i == 749
 keff_cycle , k_effective_exp, std_k_effective 1.5857267121667051   1.5992235147009979  0.0010485688106815032
i == 750
 keff_cycle , k_effective_exp, std_k_effective 1.605492652710518   1.5992349131337424  0.00104672264985891
i == 751
 keff_cycle , k_effective_exp, std_k_effective 1.6380943621701582   1.599305438449598  0.0010471987651115191
i == 752
 keff_cycle , k_effective_exp, std_k_effective 1.6200395871311204   1.5993430003131515  0.0010459746016706945
i == 753
 keff_cycle , k_effective_exp, std_k_effective 1.585969303407688   1.5993188164127798  0.0010443614798753191
i == 754
 keff_cycle , k_effective_exp, std_k_effective 1.6024014816362635   1.5993243807904396  0.0010424894966710222
i == 755
 keff_cycle , k_effective_exp, std_k_effective 1.604998

 keff_cycle , k_effective_exp, std_k_effective 1.6083175953585902   1.5993055303939907  0.0009915842986528603
i == 818
 keff_cycle , k_effective_exp, std_k_effective 1.5916875554859602   1.599293203573751  0.000990055234435086
i == 819
 keff_cycle , k_effective_exp, std_k_effective 1.568898695548601   1.5992441009759721  0.000989673359476003
i == 820
 keff_cycle , k_effective_exp, std_k_effective 1.5979495127215584   1.5992420129304006  0.0009880780290179839
i == 821
 keff_cycle , k_effective_exp, std_k_effective 1.590523520787982   1.5992279734905577  0.0009865855359318951
i == 822
 keff_cycle , k_effective_exp, std_k_effective 1.5777087995301324   1.599193376747856  0.0009856055033691247
i == 823
 keff_cycle , k_effective_exp, std_k_effective 1.604872308852884   1.599202492208699  0.0009840644198299926
i == 824
 keff_cycle , k_effective_exp, std_k_effective 1.6077683352719987   1.5992162195213002  0.0009825820222848808
i == 825
 keff_cycle , k_effective_exp, std_k_effective 1.5844282

 keff_cycle , k_effective_exp, std_k_effective 1.579116616982772   1.5991981062872151  0.0009456041249026962
i == 888
 keff_cycle , k_effective_exp, std_k_effective 1.6017366120305851   1.5992017959757956  0.0009442359089190991
i == 889
 keff_cycle , k_effective_exp, std_k_effective 1.5630863960975871   1.5991493788497024  0.0009443203703974776
i == 890
 keff_cycle , k_effective_exp, std_k_effective 1.5737822892963629   1.599112614951799  0.0009436672028449636
i == 891
 keff_cycle , k_effective_exp, std_k_effective 1.6374566967461646   1.5991681056635128  0.0009439330284730849
i == 892
 keff_cycle , k_effective_exp, std_k_effective 1.5983256707750246   1.5991668882720556  0.0009425687625521657
i == 893
 keff_cycle , k_effective_exp, std_k_effective 1.6040911386248915   1.5991739939724203  0.0009412344737767385
i == 894
 keff_cycle , k_effective_exp, std_k_effective 1.6040904082126182   1.5991810781427953  0.0009399039471106393
i == 895
 keff_cycle , k_effective_exp, std_k_effective 1.6

 keff_cycle , k_effective_exp, std_k_effective 1.619938603448616   1.5993538401920373  0.0008907888448919973
i == 958
 keff_cycle , k_effective_exp, std_k_effective 1.5996721780167393   1.5993542601627824  0.0008896129846385942
i == 959
 keff_cycle , k_effective_exp, std_k_effective 1.5616923870915371   1.5993046397766544  0.0008898247243058482
i == 960
 keff_cycle , k_effective_exp, std_k_effective 1.6221353217210448   1.5993346801476338  0.0008891607341407722
i == 961
 keff_cycle , k_effective_exp, std_k_effective 1.6041867028269825   1.5993410559987238  0.0008880144437653779
i == 962
 keff_cycle , k_effective_exp, std_k_effective 1.5930704215316698   1.599332826819633  0.0008868864837757619
i == 963
 keff_cycle , k_effective_exp, std_k_effective 1.6272203886061616   1.5993693767040191  0.000886477158709665
i == 964
 keff_cycle , k_effective_exp, std_k_effective 1.594349579650109   1.599362806289027  0.0008853404689177373
i == 965
 keff_cycle , k_effective_exp, std_k_effective 1.6171

In [77]:
first_group_flux = []
secound_group_flux = []
third_group_flux = []

number_particles = 1000
cs_total_pu_239 = [3.10, 0.975, 0.24]

for i in range(0, len(estimators_result[0].collision_sum) - 1):
    first_group_flux.append(estimators_result[0].collision_sum[i]/(cs_total_pu_239[0]))

for i in range(0, len(estimators_result[1].collision_sum) - 1):
    secound_group_flux.append(estimators_result[1].collision_sum[i]/(cs_total_pu_239[1]))

for i in range(0, len(estimators_result[2].collision_sum) - 1):
    third_group_flux.append(estimators_result[2].collision_sum[i]/(cs_total_pu_239[2]))
    

average_flux_difference_1 = []    
for i in range(0, len(first_group_flux)):
    average_flux_difference_1.append(first_group_flux[i] / secound_group_flux[i])
    
average_flux_difference_2 = []    
for i in range(1, len(first_group_flux)):
    average_flux_difference_2.append(first_group_flux[i] / third_group_flux[i])
    
average_flux_difference_3 = []    
for i in range(1, len(first_group_flux)):
    average_flux_difference_3.append(secound_group_flux[i] / third_group_flux[i])

std_flux_1 = get_std(average_flux_difference_1)
flux_difference_1 = sum(average_flux_difference_1)/len(average_flux_difference_1)

std_flux_2 = get_std(average_flux_difference_2)
flux_difference_2 = sum(average_flux_difference_2)/len(average_flux_difference_2)

std_flux_3 = get_std(average_flux_difference_3)
flux_difference_3 = sum(average_flux_difference_3)/len(average_flux_difference_3)

In [80]:
difference = (flux_difference_1 - 0.3125) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_1 * 100000))

group 1 to group 2 flux difference flux == 10.037718877420332with std   47.33748876027383


In [81]:
difference = (flux_difference_2 - 0.150) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_2 * 100000))

group 1 to group 2 flux difference flux == -13.887982091934091with std   14.276230824579795


In [82]:
difference = (flux_difference_3 - 0.480) * 100000

print('group 1 to group 2 flux difference flux == '+ str(difference)+ 'with std   '+ str(std_flux_3 * 100000))

group 1 to group 2 flux difference flux == 16.78668782030823with std   43.43375907735329


In [83]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [84]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)

In [85]:
def save_flux_to_file(estimators, file_name):
        
    k = 0
    for estimator in estimators:
        
        result_file_name = file_name + str(k)+".txt"
        
        f = open(result_file_name, "w")
        array_without_zeros = estimator.collision_sum
        for i in range(0, len(array_without_zeros)):
            f.write(" flux "+ str(array_without_zeros[i]) +'\n')
        k += 1
        
        f.close()

In [86]:
file_name = "result_flux.txt"
save_flux_to_file(estimators, file_name)
